# Week 4

Yay! It's week 4. Last week had a lot of material, this week we only have three components with very little reading. 


## Overview

* A video lecture with a few questions
* An exercise on visualizing geodata using a different set of tools from the ones we played with previously.
* Thinking about visualization, data quality, and binning. Why ***looking at the details of the data before applying fancy methods*** is often important.

## Part 1: More lecturing on dataviz

We begin today by learning more about the theory of visualization, digging into data encodings and representations.

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/zE6Nr8trdrw/0.jpg)](https://www.youtube.com/watch?v=zE6Nr8trdrw)

> *Excercise:* Some questions about the video. <font color=gray>Try to answer using your human brain (rather than your LLMs first). </font>
>
> * Mention 10 examples of ways we can encode data.
> * Are all encodings created equally? Why not? Can you think of an example from the previous lectures?
> * Mention 3 encodings that are difficult for the human eye to parse. Can you find an example of a visualization online that uses one of those three?
> * Explain in your own words: What is the problem with pie-charts?

## Part 2: Visualizing geo-data

It turns out that `plotly` (which we used during Week 2) is not the only way of working with geo-data. There are many different ways to go about it. (The more advanced PhD and PostDoc researchers in my group simply use matplotlib, since that provides more control. For an example of that kind of thing, check out [this tutorial](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621).)

Today, we'll try another library for geodata called [Folium](https://github.com/python-visualization/folium). It's good for you all to try out a few different libraries - remember that data visualization and analysis in Python is all about the ability to use many different tools. 

The exercise below is based on the code illustrated in this nice [tutorial](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data), so let us start by taking a look at that one.

*Reading*. Read through the following tutorial
 * "How to: Folium for maps, heatmaps & time data". Get it here: https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data. \[**UPDATE 2024**: Note that the Stamen tiles are no longer avialible.\]
 * (Optional) There are also some nice tricks in "Spatial Visualizations and Analysis in Python with Folium". Read it here: https://towardsdatascience.com/data-101s-spatial-visualizations-and-analysis-in-python-with-folium-39730da2adf

> *Exercise*: A different take on geospatial data. <font color=gray>It's OK to use your LLM for all of the Folium exercises</font>.
>
>A couple of weeks ago (Part 3 of Week 2), we worked with spacial data by using color-intensity of shapefiles to show the counts of certain crimes within those individual areas. Today, we look at studying geospatial data by plotting raw data points as well as heatmaps on top of actual maps.
> 
> * First start by plotting a map of San Francisco with a nice tight zoom. Simply use the command `folium.Map([lat, lon], zoom_start=13)`, where you'll have to look up San Francisco's longitude and latitude.

In [4]:
import matplotlib
import folium

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

SAVE_PATH = "plots/Week4/"

FILENAME = "data/Police_Department_Incident_Reports__Historical_2003_to_May_2018_20240130.csv"

In [5]:
crime_df = pd.read_csv(FILENAME)
crime_df = crime_df[crime_df["Date"].str.contains("2018") == False] # Remove 2018 data

print(crime_df.head())

             PdId  IncidntNum  Incident Code       Category  \
0   4133422003074    41334220           3074        ROBBERY   
1   5118535807021    51185358           7021  VEHICLE THEFT   
2   4018830907021    40188309           7021  VEHICLE THEFT   
3  11014543126030   110145431          26030          ARSON   
4  10108108004134   101081080           4134        ASSAULT   

                Descript DayOfWeek        Date   Time PdDistrict Resolution  \
0  ROBBERY, BODILY FORCE    Monday  11/22/2004  17:50  INGLESIDE       NONE   
1      STOLEN AUTOMOBILE   Tuesday  10/18/2005  20:00       PARK       NONE   
2      STOLEN AUTOMOBILE    Sunday  02/15/2004  02:00   SOUTHERN       NONE   
3                  ARSON    Friday  02/18/2011  05:27  INGLESIDE       NONE   
4                BATTERY    Sunday  11/21/2010  17:00   SOUTHERN       NONE   

   ... Fix It Zones as of 2017-11-06  2 2  DELETE - HSOC Zones 2 2  \
0  ...                                NaN                      NaN   
1  ...

In [14]:
san_fran_lat = 37.77919
san_fran_lon = -122.41914
folium.Map([san_fran_lat, san_fran_lon], zoom_start=11)


> * Next, use the the coordinates for SF City Hall `37.77919, -122.41914` to indicate its location on the map with a nice, pop-up enabled maker. (In the screenshot below, I used the black & white Stamen tiles, because they look cool. <mark>**UPDATE 2024**: Note that the Stamen tiles are no longer avialible, but there are many other tile-options. Link for more options on Stamen [**here**](https://stamen.com/here-comes-the-future-of-stamen-maps/)</mark>).
>  
> <img src="https://raw.githubusercontent.com/suneman/socialdata2022/main/files/city_hall_2022.png" alt="drawing" width="600"/>
>

In [11]:
fol_map = folium.Map([san_fran_lat, san_fran_lon], tiles="OpenTopoMap", zoom_start=13)
folium.Marker([san_fran_lat, san_fran_lon], popup="San Francisco City Hall").add_to(fol_map)

fol_map

> * Now, let's plot some more data (no need for pop-ups this time). Select a couple of months of data for `'DRUG/NARCOTIC'` and draw a little dot for each arrest for those two months. You could, for example, choose June-July 2016, but you can choose anything you like - the main concern is to not have too many points as this uses a lot of memory and makes Folium behave non-optimally. 
> We can call this kind of visualization a *point scatter plot*.

In [21]:
# Re-initialize the map
fol_map = folium.Map([san_fran_lat, san_fran_lon], tiles="OpenTopoMap", zoom_start=13)

# reformat the dataframe
crime_df['Date'] = pd.to_datetime(crime_df['Date'], format='%m/%d/%Y')
start_date = '2016-06-01'
end_date = '2016-08-01'

selected_crime = "DRUG/NARCOTIC"

# Filter data for `selected_crime` within the specified date and time range
df_filtered = crime_df[(crime_df['Category'] == selected_crime) &
                  (crime_df['Date'].between(start_date, end_date))]

for i in range(0, len(df_filtered)):
    folium.Marker([df_filtered.iloc[i]['Y'], df_filtered.iloc[i]['X']], popup=df_filtered.iloc[i]['Descript']).add_to(fol_map)

fol_map

Ok. Time for a little break. Note that a nice thing about Folium is that you can zoom in and out of the maps.

> *Exercise*: Heatmaps.
> * Now, let's play with **heatmaps**. You can figure out the appropriate commands by grabbing code from the main [tutorial](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data)) and modifying to suit your needs.
>    * To create your first heatmap, grab all arrests for the category `'SEX OFFENSES, NON FORCIBLE'` across all time. Play with parameters to get plots you like.
>    * Now, comment on the differences between scatter plots and heatmaps. 
>.      - What can you see using the scatter-plots that you can't see using the heatmaps? 
>.      - And *vice versa*: what does the heatmaps help you see that's difficult to distinguish in the scatter-plots?
>    * Play around with the various parameters for heatmaps. You can find a list here: https://python-visualization.github.io/folium/plugins.html
>    * Comment on the effect on the various parameters for the heatmaps. How do they change the picture? (at least talk about the `radius` and `blur`).

In [31]:
from folium import plugins

# Re-initialize the map
heat_san_fran = folium.Map([san_fran_lat, san_fran_lon], tiles="OpenTopoMap", zoom_start=11)

start_date = '2007-01-01'
end_date = '2017-12-31'

selected_crime = "SEX OFFENSES, NON FORCIBLE"

# Filter data for `selected_crime` within the specified date and time range
heat_df = crime_df[(crime_df['Category'] == selected_crime) &
                  (crime_df['Date'].between(start_date, end_date))]

# Ensure you're handing it floats
heat_df['X'] = heat_df['X'].astype(float)
heat_df['Y'] = heat_df['Y'].astype(float)

heat_df = heat_df[['Y', 'X']]
heat_df = heat_df.dropna(axis=0, subset=['Y','X'])

# List comprehension to make out list of lists
heat_data = [[row['Y'], row['X']] for _, row in heat_df.iterrows()]

# Plot it on the map
plugins.HeatMap(heat_data).add_to(heat_san_fran)

# Display the map
heat_san_fran

C:\Users\daflo\AppData\Local\Temp\ipykernel_30948\3351446563.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_df['X'] = heat_df['X'].astype(float)
C:\Users\daflo\AppData\Local\Temp\ipykernel_30948\3351446563.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_df['Y'] = heat_df['Y'].astype(float)


For the final element of working with heatmaps, let's now use the cool Folium functionality `HeatMapWithTime` to create a visualization of how the patterns of your favorite crime-type changes over time.

> *Exercise*: Heatmap movies. This exercise is a bit more independent than above - you get to make all the choices.
> * Start by choosing your favorite crimetype. Prefereably one with spatial patterns that change over time (use your data-exploration from the previous lectures to choose a good one).
> * Now, choose a time-resolution. You could plot daily, weekly, monthly datasets to plot in your movie. Again the goal is to find interesting temporal patterns to display. We want at least 20 frames though.
> * Create the movie using `HeatMapWithTime`.
> * Comment on your results: 
>   - What patterns does your movie reveal?
>   - Motivate/explain the reasoning behind your choice of crimetype and time-resolution. 

per hour

In [73]:
# Re-initialize the map
heat_time_san_fran = folium.Map([san_fran_lat, san_fran_lon], tiles="OpenTopoMap", zoom_start=11)

start_date = '2007-01-01'
end_date = '2017-12-31'

selected_crime = "PROSTITUTION"

# Filter data for `selected_crime` within the specified date and time range
heat_time_df = crime_df[(crime_df['Category'] == selected_crime) &
                  (crime_df['Date'].between(start_date, end_date))]

# Create weight column, using date
heat_time_df['Time'] = pd.to_datetime(crime_df['Time'])
heat_time_df['Weight'] = heat_time_df['Time'].dt.hour

time_range = range(min(heat_time_df['Weight']), max(heat_time_df['Weight']))

# List comprehension to make out list of lists
heat_data = [[[row['Y'], row['X']] for _, row in heat_time_df[heat_time_df['Weight'] == i].iterrows()] for i in time_range]

# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data ,auto_play=True, max_opacity=0.8)
hm.add_to(heat_time_san_fran)

# Display the map
heat_time_san_fran



C:\Users\daflo\AppData\Local\Temp\ipykernel_30948\2362884601.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  heat_time_df['Time'] = pd.to_datetime(crime_df['Time'])
C:\Users\daflo\AppData\Local\Temp\ipykernel_30948\2362884601.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_time_df['Time'] = pd.to_datetime(crime_df['Time'])
C:\Users\daflo\AppData\Local\Temp\ipykernel_30948\2362884601.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

Per year

In [72]:
# Re-initialize the map
heat_time_san_fran = folium.Map([san_fran_lat, san_fran_lon], tiles="OpenTopoMap", zoom_start=11)

start_date = '2005-01-01'
end_date = '2017-12-31'

selected_crime = "PROSTITUTION"

# Filter data for `selected_crime` within the specified date and time range
heat_time_df = crime_df[(crime_df['Category'] == selected_crime) &
                  (crime_df['Date'].between(start_date, end_date))]

# Ensure you're handing it floats
heat_time_df['X'] = heat_time_df['X'].astype(float)
heat_time_df['Y'] = heat_time_df['Y'].astype(float)

heat_time_df = heat_time_df[['Y', 'X']]

# Create weight column, using date
heat_time_df['Weight'] = crime_df['Date'].dt.year
heat_time_df = heat_time_df.dropna(axis=0, subset=['Y', 'X', 'Weight'])

time_range = range(min(heat_time_df['Weight']), max(heat_time_df['Weight']))

# List comprehension to make out list of lists
heat_data = [[[row['Y'], row['X']] for _, row in heat_time_df[heat_time_df['Weight'] == i].iterrows()] for i in time_range]

# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data ,auto_play=True, max_opacity=0.8)
hm.add_to(heat_time_san_fran)

# Display the map
heat_time_san_fran

C:\Users\daflo\AppData\Local\Temp\ipykernel_30948\352499711.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_time_df['X'] = heat_time_df['X'].astype(float)
C:\Users\daflo\AppData\Local\Temp\ipykernel_30948\352499711.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_time_df['Y'] = heat_time_df['Y'].astype(float)


## Part 3: Errors in the data. The importance of looking at raw (or close to raw) data.

We started the course by plotting simple histogram and bar plots that showed a lot of cool patterns. But sometimes the binning can hide imprecision, irregularity, and simple errors in the data that could be misleading. In the work we've done so far, we've already come across at least three examples of this in the SF data. <font color=gray>It's 100% OK to use your LLM for this one.</font>

1. In the temporal activity for `PROSTITUTION` something surprising is going on on Thursday. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/prostitution.png), where I've highlighted the phenomenon I'm talking about.
2. Last week, when we investigated the details of how the timestamps are recorded using jitter-plots in the DAOST exercises, we saw that many more crimes were recorded e.g. on the hour, 15 minutes past the hour, and to a lesser in whole increments of 10 minutes. Crimes didn't appear to be recorded as frequently in between those round numbers. Remind yourself [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/jitter.png), where I've highlighted the phenomenon I'm talking about.
3. Also, the *Hall of Justice* on the 800 block of Bryant street seems to be an unlikely hotspot for sex offences. Take a look here [**here**](https://raw.githubusercontent.com/suneman/socialdata2022/main/files/crime_hot_spot.png).

> *Exercise*: Data errors. The data errors we discovered above become difficult to notice when we aggregate data (and when we calculate mean values, as well as statistics more generally). Thus, when we visualize, errors become difficult to notice when binning the data. We explore this process in the exercise below.
>
>This last exercise for today has two parts:
> * In each of the examples above, describe in your own words how the data-errors I call attention to above can bias the binned versions of the data. Also, briefly mention how not noticing these errors can result in misconceptions about the underlying patterns of what's going on in San Francisco (and our modeling).
> * Find your own example of human noise in the data and visualize it.
> * Were you able to use your LLM for anything in this exercise?